In [59]:
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.io import curdoc
from bokeh.layouts import layout
from bokeh.models import (Button, CategoricalColorMapper, ColumnDataSource,
                          HoverTool, Label, SingleIntervalTicker, Slider,)
from bokeh.palettes import Spectral6
from bokeh.plotting import figure

import pandas as pd
import numpy as np

df = pd.read_csv("gapminder.csv")    
years = df['year'].unique()

# Turn population into bubble sizes. Use min_size and factor to tweak.
scale_factor = 200
pop_size = (np.sqrt(df['pop'].values.astype(float) / np.pi) / scale_factor).astype(int)
min_size = 3
pop_size[np.where(pop_size < min_size)] = min_size
df['popScale'] = pop_size

def make_document(doc):

    plot = figure(x_range=(0, 40000), y_range=(25, 90), title='Gapminder Data', plot_height=300)
    plot.xaxis.ticker = SingleIntervalTicker(interval=5000)
    plot.xaxis.axis_label = "GDP per capita"
    plot.yaxis.ticker = SingleIntervalTicker(interval=20)
    plot.yaxis.axis_label = "Life expectancy"

    label = Label(x=1.1, y=23, text=str(years[0]), text_font_size='93px', text_color='#eeeeee')
    plot.add_layout(label)

    is_year = df['year'] == years[0]
    source = ColumnDataSource(data=df[is_year])

    color_mapper = CategoricalColorMapper(palette=Spectral6, factors=df['continent'].unique())
    plot.circle(
        x='gdpPercap',
        y='lifeExp',
        size='popScale',
        source=source,
        fill_color={'field': 'continent', 'transform': color_mapper},
        fill_alpha=0.8,
        line_color='#7c7e71',
        line_width=0.5,
        line_alpha=0.5,
        legend_group='continent',
    )
    plot.add_tools(HoverTool(tooltips="@country", show_arrow=False, point_policy='follow_mouse'))


    def animate_update():
        year = slider.value + 5
        if year > years[-1]:
            year = years[0]
        slider.value = year


    def slider_update(attrname, old, new):
        year = slider.value
        label.text = str(year)
        is_year = df['year'] == year
        source.data = df[is_year]

    slider = Slider(start=years[0], end=years[-1], value=years[0], step=1, title="Year")
    slider.on_change('value', slider_update)

    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update, 200)
        else:
            button.label = '► Play'
            curdoc().remove_periodic_callback(callback_id)

    button = Button(label='► Play', width=60)
    button.on_click(animate)

    doc.add_root(button)
    doc.add_root(slider)
    doc.add_root(plot)

apps = {'/': Application(FunctionHandler(make_document))}

server = Server(apps, port=5003)
server.start()

In [58]:
server.stop()